In [ ]:
from qat.lang.AQASM import Program, QRoutine, RZ, RX, CNOT, H, X, Z
from qat.core import Observable, Term
from qat.qpus import get_default_qpu, PyLinalg
from qat.plugins import ScipyMinimizePlugin
from math import sqrt

from pysat.formula import CNF

# Resolvendo o problema 3-SAT usando Grover

<!-- Descrever Grover -->
O algoritmo de Grover é um algoritmo quântico de busca que pode ser usado para encontrar soluções em problemas não estruturados com um ganho quadrático em relação a suas contrapartes clássicas.

<!-- Descrever 3-SAT -->
O Problema de Satisfatibilidade Booleana é o problema de determinar se existe uma interpretação que satisfaça uma fórmula Booleana. Ou seja, o problema busca uma sequência de valores VERDADEIRO ou FALSO que possa substituir as variáveis da fórmula Booleana de forma que seu valor final seja VERDADEIRO. Caso exista essa sequência, a fórmula é *satisfazível*. Por outro lado, se não existe essa sequência, a expressão definida pela fórmula é FALSO para todos os valores e é *insatisfazível*. Esse problema pode ser visto como uma busa, onde a solução é a sequência que satisfaz a fórmula booleana.

<!-- Usando Grover em 3-SAT -->
Para *buscas não-estruturadas*, o Algoritmo de Grover é ótimo e possui complexidade de $O(\sqrt{N}) = O(2^{\frac{n}{2}}) = O(1.414^n)$.
Para o problema do 3-SAT, o melhor algoritmo clássico conhecido possui a complexidade de $O(1.307^n)$. Apesar de o algoritmo de Grover poder ser usado para acelerar soluções de problemas NP-Completos, esses problemas possuem uma estrutura, que pode ser aproveitada para se obter complexidades melhores do que a do algoritmo de Grover.
Apesar de não fazer sentido usar Grover para resolver 3-SAT, as técnicas usadas aqui podem ser expandidas para casos gerais, como o k-SAT, no qual Grover supera o melhor algoritmo clássico atual.

<!-- Grover não é busca em um banco de dados -->
Quando usa-se Grover para resolver um problema é comum confundir o seu comportamento. Em vez de ser interpretado como uma busca em um banco de dados, Grover é capaz de encontrar uma variável $x_0$ que seja capaz de resolver uma função $f$ específica.

A função $f$ é uma função booleana arbitrária $f:\big\{0,1\big\}^n\rightarrow \{0,1\}$. Para aplicações como quebra de criptografia, seu comportamento não é uma "busca em um banco de dados", em que seria necessário armazenar todo o banco de chaves no circuito quântico, mas na verdade uma função do tipo

$$\begin{equation*}
    f(x) = \left\{
  \begin{array}{ll}
    1, & \hbox{se ${x=x_0}$,} \\
    0, & \hbox{caso contrário.}
  \end{array}
\right.
\end{equation*}$$

This seems to be a common misunderstanding about Grover's algorithm. It is not about querying a magically encoded database. Rather, you have an efficiently computable function $f(x) \in {0,1}$ and you want to find some $x_0$ for which $f(x_0)=1$. Since you know how to realize $f(x)$ (i.e., you have a circuit), you can run f on a quantum computer and use Grover to find such an x0. This function can be seen as returning entries of a "database", which is encoded in a specific function, though I don't particularly like this picture.

The relevance is in the fact that a large number of interesting problems (namely, the class NP) are such that solutions might be hard to find, but they are easy to verify. Thus, Grover gives a square-root speed-up on any brute-force method to solve such a problem (i.e., any method which does not make use of any special structural property of f).

In [ ]:
# remove one of the clauses
cnf = CNF(from_file='example.cnf')
num_literals = cnf.nv

print(cnf.clauses)
print(num_literals)


In [ ]:
def oracle(num_qubits, cnf):
    routine = QRoutine()
    # Edite aqui o seu código


    return routine

In [ ]:
def diffusion(num_qubits):
    routine = QRoutine()
    wires = routine.new_wires(num_qubits)
    # Apply transformation |s> -> |00..0> (H-gates)
    for qubit in wires:
        H(qubit)
    # Apply transformation |00..0> -> |11..1> (X-gates)
    for qubit in wires:
        X(qubit)
    # Do multi-controlled-Z gate
    H(wires[0])
    X.ctrl(len(wires)-1)(wires[1:], wires[0])
    H(wires[0])
    # Apply transformation |11..1> -> |00..0>
    for qubit in wires:
        X(qubit)
    # Apply transformation |00..0> -> |s>
    for qubit in wires:
        H(qubit)

    return routine

In [ ]:
def check_result(num_qubits):
    routine = QRoutine()
    wires = routine.new_wires(num_qubits)
    ancilla = routine.new_wires(len(cnf.clauses)+1)
    routine.set_ancillae(ancilla)

    for i, clause in enumerate(cnf.clauses):
        qubits = [abs(q) for q in clause]
        for control in clause:
            if control > 0:
                X(wires[abs(control)-1])
        X.ctrl(len(qubits))([wires[n-1] for n in qubits], ancilla[i])
        for control in clause:
            if control > 0:
                X(wires[abs(control)-1])

    for qubit in ancilla[:-1]:
        X(qubit)
    X.ctrl(len(ancilla)-1)(ancilla[:-1], ancilla[-1])
    for qubit in ancilla[:-1]:
        X(qubit)
    Z(ancilla[-1])

    return routine

In [ ]:
qprog = Program()
num_qubits = num_literals
# --------------------------
# Initial state preparation
# --------------------------
qbits = qprog.qalloc(num_qubits)
cbits = qprog.calloc(num_qubits)
for q in qbits:
    H(q)

# ----------------------------------
# Alternate application of operators
# ----------------------------------
oracle_operator = oracle(num_qubits, cnf)
diffusion_operator = diffusion(num_qubits)
check = check_result(num_qubits)

for step in range(int(sqrt(num_qubits))):
    oracle_operator(qbits)
    diffusion_operator(qbits)
check(qbits)

### Visualização do Circuito

In [ ]:
circuit = qprog.to_circ()
print("total number of gates: ", len(circuit.ops))
print("Variables:", circuit.get_variables())
# Display quantum circuit
%qatdisplay circuit --svg

In [ ]:
# Create a job
job = circuit.to_job(nbshots=1000)

qpu = PyLinalg()

# Submit the job to the QPU
result = qpu.submit(job)

# Iterate over the final state vector to get all final components
for sample in result:
     print("State %s: probability %s +/- %s amplitude %s " % (sample.state, sample.probability, sample.err, sample.amplitude))